In [ ]:
!pip3 install --upgrade pip
!pip3 install tensorflow==2.7.0
!pip3 install -U tensorflow-quantum 
!pip3 install seaborn matplotlib

!pip3 install googleapis-common-protos==1.52.0 google-api-core==1.21.0 google-auth==1.18.0 google-api-python-client==1.8.0 grpcio==1.34.1 protobuf==3.17.3

  Obtaining dependency information for MarkupSafe>=2.1.1 from https://files.pythonhosted.org/packages/de/e2/32c14301bb023986dff527a49325b6259cab4ebb4633f69de54af312fc45/MarkupSafe-2.1.3-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached MarkupSafe-2.1.3-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.0 kB)
Using cached MarkupSafe-2.1.3-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (25 kB)
  Attempting uninstall: MarkupSafe
    Found existing installation: MarkupSafe 2.0.0
    Not uninstalling markupsafe at /shared-libs/python3.8/py-core/lib/python3.8/site-packages, outside environment /root/venv
    Can't uninstall 'MarkupSafe'. No files were found to uninstall.


In [ ]:
#!pip3 install cirq-core==0.13.1 cirq-google==0.13.1 sympy==1.8 numpy==1.19.5 nbformat==4.4.0 pylint==2.4.4 yapf==0.28.0 tensorflow==2.7.0 googleapis-common-protos==1.52.0 google-api-core==1.21.0 google-auth==1.18.0 google-api-python-client==1.8.0 grpcio==1.34.1 protobuf==3.17.3 

  Using cached nbformat-4.4.0-py2.py3-none-any.whl (155 kB)
  Obtaining dependency information for MarkupSafe>=2.1.1 from https://files.pythonhosted.org/packages/de/e2/32c14301bb023986dff527a49325b6259cab4ebb4633f69de54af312fc45/MarkupSafe-2.1.3-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached MarkupSafe-2.1.3-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.0 kB)
Using cached MarkupSafe-2.1.3-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (25 kB)
  Attempting uninstall: MarkupSafe
    Found existing installation: MarkupSafe 2.0.0
    Not uninstalling markupsafe at /shared-libs/python3.8/py-core/lib/python3.8/site-packages, outside environment /root/venv
    Can't uninstall 'MarkupSafe'. No files were found to uninstall.
  Attempting uninstall: nbformat
    Found existing installation: nbformat 5.7.0
    Not uninstalling nbformat at /shared-libs/python3.8/py-core/lib/python3.8/site-packages, outside environment /root/venv
  

In [ ]:
# Update package resources to account for version changes.
import importlib, pkg_resources
importlib.reload(pkg_resources)

<module 'pkg_resources' from '/root/venv/lib/python3.8/site-packages/pkg_resources/__init__.py'>

In [ ]:
import tensorflow as tf
import tensorflow_quantum as tfq

import cirq
import sympy
import numpy as np
import seaborn as sns
import collections

# visualization tools
%matplotlib inline
import matplotlib.pyplot as plt
from cirq.contrib.svg import SVGCircuit

NotFoundError: /usr/local/lib/python3.8/site-packages/tensorflow_quantum/core/ops/_tfq_simulate_ops.so: cannot open shared object file: No such file or directory

In [ ]:
# Load the ImageNet dataset from TensorFlow's built-in datasets
(train_ds, val_ds), metadata = tf.keras.datasets.imagenet.load_data(
    split=["train", "validation"],
    as_supervised=True,
    with_info=True
)
# Preprocess the dataset
batch_size = 48
img_height = 150
img_width = 150

# Resize and normalize images
def preprocess(image, label):
    image = tf.image.resize(image, (img_height, img_width))
    image = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)(image)
    return image, label

train_ds = train_ds.map(preprocess).batch(batch_size)
val_ds = val_ds.map(preprocess).batch(batch_size)

# Data augmentation (you can customize this)
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomZoom(0.1),
    ]
)

In [ ]:
# Create a quantum convolution layer using TFQ
def create_quantum_conv_layer(num_qubits, num_filters):
    return tfq.layers.PQC(
        model_circuit=create_quantum_circuit(num_qubits, num_filters),
        observables=['x' * num_qubits]
    )

# Create a quantum circuit for the convolution operation
def create_quantum_circuit(num_qubits, num_filters):
    qubits = cirq.GridQubit.rect(1, num_qubits)
    circuit = cirq.Circuit()
    for i, qubit in enumerate(qubits):
        for _ in range(num_filters):
            circuit.append(cirq.H(qubit))
    return circuit

model = keras.Sequential([
    data_augmentation,
    create_quantum_conv_layer(4, 16),  # Adjust num_qubits and num_filters as needed
    layers.Flatten(),
    layers.Dense(500, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Dense(500, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Dense(1000, activation='softmax')  # Number of classes in ImageNet
])

# Adjust the optimizer, loss, and metrics as needed
opt = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(
    opt,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=5)
check_point = tf.keras.callbacks.ModelCheckpoint('model_{}.h5'.format('12'),
                                                 monitor='accuracy',
                                                 save_best_only=True)

epochs = 50
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    callbacks=[early_stop, check_point]
)

fig, axs = plt.subplots(2, 1, figsize=(15, 15))
axs[0].plot(history.history['loss'])
axs[0].plot(history.history['val_loss'])
axs[0].set_title('Training Loss vs Validation Loss')
axs[0].set_xlabel('Epochs')
axs[0].set_ylabel('Loss')
axs[0].legend(['Train', 'Val'])
axs[1].plot(history.history['accuracy'])
axs[1].plot(history.history['val_accuracy'])
axs[1].set_title('Training Accuracy vs Validation Accuracy')
axs[1].set_xlabel('Epochs')
axs[1].set_ylabel('Accuracy')
axs[1].legend(['Train', 'Val'])

plt.show()


2023-09-18 05:45:12.325626: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-18 05:45:12.540738: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-09-18 05:45:12.540771: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-09-18 05:45:12.604804: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-09-18 05:45:14.690173: W tensorflow/stream_executor/pla

ModuleNotFoundError: No module named 'tensorflow_quantum'

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=4029068e-263e-4c12-9932-d3ac72036324' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>